In [24]:
# Importing the packages
import pandas as pd
import sqlite3
from function_list import write_dfs_to_sql

In [25]:
# Reading in the all the CSVs
df_active_zone_minutes = pd.read_csv("cleaned_daily_active_zone_minutes.csv")
df_daily_heart_rate = pd.read_csv("cleaned_daily_heart_rate.csv")
df_steps = pd.read_csv("cleaned_daily_steps.csv")
df_sleep = pd.read_csv("cleaned_sleep.csv")

In [26]:
# Create the SQL database
conn = sqlite3.connect("fitbit_data.db")

In [27]:
# Creating a dictionary and using write_dfs_to_sql function to add each CSV to fitbit_data database
df_dict = {
    'active_zone_minutes': df_active_zone_minutes,
    'heart_rate': df_daily_heart_rate,
    'steps': df_steps,
    'sleep': df_sleep
}

write_dfs_to_sql(df_dict, conn)

In [28]:
# Creating query to combine all the data for when the sleep_info_code is 0 (Sufficient data to generate a sleep log)
query = '''
SELECT 
    azm.date,
    azm.active_zone_minutes,
    hr.avg_beats_per_minute,
    st.total_steps_taken,
    sl.*
FROM active_zone_minutes azm
JOIN heart_rate hr 
    ON azm.date = hr.date
JOIN steps st 
    ON azm.date = st.date
JOIN sleep sl 
    ON azm.date = sl.date_of_sleep
WHERE sl.sleep_info_code = 0;
'''

In [29]:
# Running the query and creating a new dataframe
df_final = pd.read_sql_query(query, conn)

In [30]:
# Checking the first 5 rows of the new dataframe
df_final.head()

,date,active_zone_minutes,avg_beats_per_minute,total_steps_taken,date_of_sleep,sleep_start_time,sleep_end_time,sleep_duration,minutes_asleep,minutes_awake,...,sleep_deep_thirtyDayAvgMinutes,sleep_wake_count,sleep_wake_minutes,sleep_wake_thirtyDayAvgMinutes,sleep_light_count,sleep_light_minutes,sleep_light_thirtyDayAvgMinutes,sleep_rem_count,sleep_rem_minutes,sleep_rem_thirtyDayAvgMinutes
0,2025-01-04,37,97.424612,5615,2025-01-04,2025-01-04T23:38:30.000,2025-01-05T08:51:30.000,33180000,509,44,...,0.0,19.0,44.0,0.0,24.0,261.0,0.0,6.0,151.0,0.0
1,2025-01-05,22,76.008063,8105,2025-01-05,2025-01-06T00:08:30.000,2025-01-06T07:54:00.000,27900000,409,56,...,97.0,19.0,56.0,44.0,20.0,228.0,261.0,10.0,122.0,151.0
2,2025-01-06,64,74.742099,5923,2025-01-06,2025-01-07T00:14:30.000,2025-01-07T07:41:30.000,26820000,397,50,...,78.0,15.0,50.0,50.0,17.0,263.0,245.0,4.0,95.0,137.0
3,2025-01-07,79,84.219651,20018,2025-01-07,2025-01-07T23:51:30.000,2025-01-08T07:53:30.000,28920000,417,65,...,65.0,25.0,65.0,50.0,21.0,227.0,251.0,9.0,85.0,123.0
4,2025-01-08,7,80.420794,13712,2025-01-08,2025-01-08T23:15:30.000,2025-01-09T07:24:30.000,29340000,445,44,...,75.0,14.0,44.0,54.0,16.0,198.0,245.0,5.0,139.0,113.0


In [31]:
# Checking the shape of the new dataframe
df_final.shape

(162, 34)

In [32]:
# Checking the info of the new dataframe
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   date                             162 non-null    object 
 1   active_zone_minutes              162 non-null    int64  
 2   avg_beats_per_minute             162 non-null    float64
 3   total_steps_taken                162 non-null    int64  
 4   date_of_sleep                    162 non-null    object 
 5   sleep_start_time                 162 non-null    object 
 6   sleep_end_time                   162 non-null    object 
 7   sleep_duration                   162 non-null    int64  
 8   minutes_asleep                   162 non-null    int64  
 9   minutes_awake                    162 non-null    int64  
 10  minutes_after_wakeup             162 non-null    int64  
 11  time_in_bed                      162 non-null    int64  
 12  sleep_efficiency      

In [33]:
# Create CSV for the new dataframe
df_final.to_csv("cleaned_total_data.csv", index=False) 

In [34]:
# Closing the connection
conn.close()